In [1]:
import deap
import pandas as pd
import yfinance as yf

In [2]:
from deap import creator
from deap import base, tools
import random

In [3]:
creator.create("FitnessMax", base.Fitness, weigths = (1.0,))
creator.create("Individual", list, fitness = creator.FitnessMax)

ind_size =10

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual",tools.initRepeat, creator.Individual, toolbox.attr_float, n= 10)


In [4]:
toolbox.individual

functools.partial(<function initRepeat at 0x0000011B7658D240>, <class 'deap.creator.Individual'>, functools.partial(<built-in method random of Random object at 0x0000011B51012550>), n=10)

In [5]:
from deap import gp, base
import operator

In [6]:
pset = gp.PrimitiveSet("MAIN", arity=2)

pset.addPrimitive(operator.add, 2)

creator.create("FitnessMin", base.Fitness, weights = (-1,))
creator.create("Individual", gp.PrimitiveTree, fitness= creator.FitnessMin, pset = pset)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset = pset, min_=-1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)

c:\Users\khann\Documents\Data Science and Financial Technology\Final project\GP_trading_strategies\.venv\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [12]:
ind1 = toolbox.individual()
ind1.

()

In [48]:
btc_ts = yf.Ticker("BTC-USD").history(start='2023-01-01', end='2023-01-20')

#Moving average function:
def ma(df:pd.DataFrame, period : int|float):
    return df.iloc[-period:].mean()

pset.addPrimitive(name="MovingAverage",primitive=ma,arity=2 )

In [51]:
pset.arguments

['ARG0', 'ARG1']

In [13]:
import random
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt


In [14]:
# Generate a synthetic dataset
np.random.seed(42)
data = pd.DataFrame({
    'price': np.random.randn(100).cumsum() + 100  # Simulated price data
})

data['short_mavg'] = data['price'].rolling(window=10, min_periods=1).mean()
data['long_mavg'] = data['price'].rolling(window=50, min_periods=1).mean()
data = data.dropna()


# Using logical operators:

In [20]:
from deap import gp

In [ ]:
pset = gp.PrimitiveSet("MAIN",2 )

#Add the logcical operators:
pset.addPrimitive(operator.lt,2) #<
pset.addPrimitive(operator.gt,2) #>

pset.addTerminal(True)
pset.addTerminal(False)

In [61]:
#Functions to apply to data:
#Assume data provided to the function is a pd DataFrame with the last entry being todays data.

#Moving average function:
def ma(df:pd.DataFrame, period : int|float= random.randint(0,10)):
    """Get the Moving average for a time period."""
    return {"value":df.iloc[-period:].mean(),"period":period}

def lag(df:pd.DataFrame, lag : int|float = random.randint(0,10)):
    """Get the lagged value from the dataset."""
    return {"value":df.iloc[-lag], "peroid":lag}

def multiply_by_constant(df:pd.DataFrame, constant : int|float = random.random()):
    """Return the last entry in the datframne multiplied by a constant between 0 and 1."""
    return {"value":df.iloc[-1]*constant,"period":constant}

In [62]:
btc_ts = yf.Ticker("BTC-USD").history(start='2023-01-01', end='2023-01-20')
btc_ts_open = btc_ts['Open']
btc_ts_open.head(5)


Date
2023-01-01 00:00:00+00:00    16547.914062
2023-01-02 00:00:00+00:00    16625.509766
2023-01-03 00:00:00+00:00    16688.847656
2023-01-04 00:00:00+00:00    16680.205078
2023-01-05 00:00:00+00:00    16863.472656
Name: Open, dtype: float64

In [75]:
# Create the tree

list_functioins = [ma,lag,multiply_by_constant]
list_functioins


[<function __main__.ma(df: pandas.core.frame.DataFrame, period: int | float = 6)>,
 <function __main__.lag(df: pandas.core.frame.DataFrame, lag: int | float = 9)>,
 <function __main__.multiply_by_constant(df: pandas.core.frame.DataFrame, constant: int | float = 0.7494776111435393)>]

In [72]:
def comparitor(func1, func2):
    return func1>func2

<function __main__.lag(df: pandas.core.frame.DataFrame, lag: int | float = 9)>

# DEAP logical tree tests

In [4]:
from deap import gp, creator, base, tools
import operator

In [19]:
# Logical operators:
pset = gp.PrimitiveSetTyped("main",[float,float],bool)
pset.addPrimitive(operator.add, [float,float],float)
pset.addPrimitive(operator.lt, [float,float],bool)
pset.addPrimitive(operator.and_, [bool,bool],bool)
pset.addPrimitive(operator.or_,[bool,bool],bool)

pset.addTerminal(1,bool)
pset.addTerminal(0,bool)

In [20]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

c:\Users\khann\Documents\Data Science and Financial Technology\Final project\GP_trading_strategies\.venv\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\khann\Documents\Data Science and Financial Technology\Final project\GP_trading_strategies\.venv\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [21]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.expr)

In [27]:
from deap import base, creator, gp

# pset = gp.PrimitiveSet("MAIN", 1)
# pset.addPrimitive(operator.add, 2)
# pset.addPrimitive(operator.sub, 2)
# pset.addPrimitive(operator.mul, 2)
# pset.renameArguments(ARG0='x')

# creator.create("Individual", gp.PrimitiveTree)

# toolbox = base.Toolbox()
# toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
# toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)

expr = toolbox.individual()
nodes, edges, labels = gp.graph(expr)

### Graphviz Section ###
import pygraphviz as pgv

g = pgv.AGraph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
g.layout(prog="dot")

for i in nodes:
    n = g.get_node(i)
    n.attr["label"] = labels[i]

g.draw("tree.png")

IndexError: The gp.generate function tried to add a primitive of type '<class 'float'>', but there is none available.